[2025-09-05 Fiddler](https://thefiddler.substack.com/p/how-low-or-high-can-you-go)
--------------------

Fiddler
-------
Without loss of generality, assume $x_1 > 1/2$ and normalize with a factor of 2.

1. The probability that $x_2 < 1/2$ is $1/2$.
2. The probability that $1/2 < x_2 < x_1$ is $x_1-1/2$.
3. The probability that $x_2 > x_1$ is $1-x_1$.

The probability of winning a second point in the first case is $1-x_2$.

The probability of winning a second point in the second case is $x_2$.

The game is over in the third case.

Integrating over $x_1$ and $x_2$,

$$ p_2 = 2\int_{1/2}^1 dx_1\,\left(\int_0^{1/2} dx_2\,(1-x_2) + \int_{1/2}^{x_1} dx_2\, x_2\right) = 13/24 $$

In [1]:
x1,x2 = var('x1,x2')

p2 = 2*integral(integral(1-x2, x2, 0, 1/2)
                + integral(x2, x2, 1/2, x1),
                x1, 1/2, 1)
p2, numerical_approx(p2)

(13/24, 0.541666666666667)

### Numerical simulations
[Numerical simulations](20250905.go) agree

    $ go run 20250905.go
    43326720/80000000=0.541584

Extra credit
------------
I'll assume my friend uses the same strategy of high when the previous number is
less than 0.5 and low when the previous number is greater then 0.5.

Let $z = |x - 0.5|$ be the difference between the current number and 0.5.

The probability that my friend wins the next round is $0.5+z$.  The problem is
then to determine $f(z)$, the probability distribution of $z$ after many successful
rounds.  The probability that my friend wins the next round is then
$p_\infty = 1/2 + \int_0^{1/2} dz\, z\,f(z)$.

Intuitively, I'd expect $f(z)$ to be higher near $z = 0$ and lower near $z = 1/2$,
since, given a current $z$, the probability density of the next $z$ less than the
current $z$ is twice the probability density for the next $z$ greater than the
current $z$.

Given the current number with $z_0$, let $F(z_0, z)$ be the probability distribution
of $z$ for the next number after a successful round.

$$ F(z_0,z) = cH(z-z_0) + 2cH(z_0-z) $$

Normalizing, $c(1/2-z_0) + 2cz_0 = 1$, $c = 2/(1 + 2z_0)$.

Let $f_n(z)$ be the probability distribution going into the $n$-th round, where $f_1 = 2$.

The probability of winning the $n$-th round is $p_n = 1/2 + \int_0^{1/2} dz\,z\,f_n(z)$.

The probability distribution going into the $n+1$-th round, is

$$ f_{n+1}(z) = \int_0^{1/2} dz_0\,f_n(z_0)\,F(z_0,z) =
        \int_0^z dz_0\,\frac{2f_n(z_0)}{1+2z_0}
        + \int_z^{1/2} dz_0\,\frac{4f_n(z_0)}{1+2z_0}
$$

In [2]:
def p(f):
    x = var('x')
    return 1/2 + integral(x*f(x), x, 0, 1/2)

def fnext(f):
    x,z = var('x,z')
    assume(z > 0, z < 1/2)
    fn(z) = integral(2*f(x)/(1+2*x),x,0,z) + integral(4*f(x)/(1+2*x),x,z,1/2)
    return fn

In [3]:
f1(z) = 2
f = f1
for i in range(20):
    print(f"p_{i+1}={numerical_approx(p(f))}")
    f = fnext(f)

p_1=0.750000000000000
p_2=0.721573590279973
p_3=0.721126555578637
p_4=0.721339984110659
p_5=0.721349833591397
p_6=0.721347664656588
p_7=0.721347496489324
p_8=0.721347518142363
p_9=0.721347520676102
p_10=0.721347520477702
p_11=0.721347520442498
p_12=0.721347520444034
p_13=0.721347520444495
p_14=0.721347520444487
p_15=0.721347520444481
p_16=0.721347520444481
p_17=0.721347520444481
p_18=0.721347520444481
p_19=0.721347520444481
p_20=0.721347520444481


Since it seems to converge to some steady-state,

$$ f_\infty = \int_0^z dz_0\,\frac{2f_\infty}{1+2z_0} + \int_z^{1/2} dz_0\,\frac{4f_\infty}{1+2z_0} $$

$$ \frac{df_\infty}{dz} = \frac{2f_\infty}{1+2z} - \frac{4f_\infty}{1+2z} = - \frac{2f_\infty}{1+2z} $$

and the solution, with the appropriate normalization constant, is

$$ f_\infty(z) = \frac{2}{\log2}\frac{1}{1+2z} $$

and the probability that my friend wins the next round is

$$ p_\infty = \frac12 + \frac{2}{\log2}\int_0^{1/2} \frac{z\,dz}{1+2z} = \frac{1}{\log4} \approx 0.72134752 $$

Of course, if my friend isn't following the strategy, which is possible, given that
the _only_ information I have is that my friend has won many rounds, then I can't
tell what the probability is.

In [4]:
integral(2/log(2)/(1+2*x), x, 0, 1/2)

1

In [5]:
p = 1/2 + integral(x*2/log(2)/(1+2*x), x, 0, 1/2)
p.simplify_full(), numerical_approx(p)

(1/2/log(2), 0.721347520444482)

### Numerical simulations
[Numerical simulations](20250905.go) seem to agree.

    $ go run 20250905.go ec
    43326720/80000000=0.541584
    0 80000000 2.693792 0.750073
    1 60005879 2.591371 0.722223
    2 43337627 2.588048 0.721132
    3 31252151 2.588868 0.721362
    4 22544123 2.588860 0.721359
    5 16262411 2.588863 0.721176
    6 11728060 2.589780 0.721445
    7 8461156 2.589710 0.721419
    8 6104037 2.589746 0.721446
    9 4403731 2.589661 0.721555
    10 3177535 2.589000 0.721242
    11 2291773 2.589639 0.721175
    12 1652769 2.590861 0.721432
    13 1192360 2.591277 0.721176
    14 859901 2.593129 0.721356
    15 620295 2.594796 0.721592
    16 447600 2.595932 0.722833
    17 323540 2.591330 0.722758
    18 233841 2.585338 0.722427
    19 168933 2.578685 0.720238
    20 121672 2.580323